# 4100 Assignment 3: Bayesian Tomatoes

In this assignment, you’ll implement a “sentiment analysis” classifier that looks at sentences and decides how positive or negative the speaker is feeling. In fact, you’ll implement two: a Naive Bayes classifier that uses each word as a piece of independent evidence, and classifier that treats each word as dependent on the last. This assignment will also serve as your introduction to machine learning.

The data we’ll be using consists of sentences pulled from the Rotten Tomatoes movie review aggregator website.  In this dataset, someone has gone through each sentence and labeled the sentiment as belonging to one of 5 categories:

0 - negative

1 - somewhat negative

2 - neutral

3 - somewhat positive

4 - positive

(In fact, the dataset has also labeled phrases within the sentence with these sentiments, but the loader will just skip anything that isn't a full sentence.)
This dataset, known as SST1, can be found on Canvas in the same place where you found this notebook.  The following code will upload it into a "dataframe" (annotated multidimensional array) if you select the file at the prompt.

In [3]:
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_table(io.BytesIO(uploaded['train.tsv']))

Saving train.tsv to train.tsv


You can see some of the dataframe's contents by just evaluating it at the prompt.  (You can also just open .tsv files as text files on your local machine.)

In [4]:
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


The following class's methods do all the necessary counting to transform this data into counts that you can use to estimate probabilities for a Naive Bayes classifier.  Its function for breaking down sentences into tokens is also pulled out for your later use.

In [6]:
# NLTK is a great toolkit that makes Python a nice choice for natural language processing (NLP)
# Note that bigrams() returns an *iterator* and not a *list* - you would need to call it
# again or store as a list to iterate over the bigrams multiple times

# This statement needed to avoid erroring out.
!pip3 install nltk==3.4

from nltk.util import bigrams

def tokenize(sentence):
    """ Returns list of tokens (strings) from the sentence.

    Sets to lowercase and separates tokens by whitespace.

    Args:
        sentence (string):  the string to tokenize
    """
    return [t.lower() for t in sentence.split()]

class ModelInfo:
    """ Contains all counts from the data necessary to do Naive Bayes.

    Attributes:
        word_counts (List[Dict[string,int]]):  counts of tokens, indexed by class
        bigram_counts (List[Dict[string,int]]): as word_counts, but for bigrams
        sentiment_counts (List[int]):  counts of sentences with each sentiment
        total_words (List[int]):  counts of words in each sentiment
        bigram_denoms (List[Dict[string,int]]):  counts of how often a token starts a bigram,
                                                 again one per sentiment.
        total_bigrams (List[int]): counts of total bigrams for each sentiment
        total_examples (int):  total sentence count
    """

    def __init__(self):
        self.word_counts = [{}, {}, {}, {}, {}]
        self.bigram_counts = [{}, {}, {}, {}, {}]
        self.sentiment_counts = [0, 0, 0, 0, 0]
        self.total_words = [0, 0, 0, 0, 0]
        self.bigram_denoms = [{}, {}, {}, {}, {}]
        self.total_bigrams = [0, 0, 0, 0, 0]
        self.total_examples = 0

    def __str__(self):
        words = self.total_words
        sentiments = self.sentiment_counts
        return "words by sentiment: {} sentences: {}".format(words, sentiments)

    def update_word_counts(self, sentence, sentiment):
        """ Consume a sentence and update all counts.

        To "tokenize" the sentence we'll make use of NLTK, a widely-used Python natural language
        processing (NLP) library.  This will handle otherwise onerous tasks like separating periods
        from their attached words.  (Unless the periods are decimal points ... it's more complex
        than you might think.)  The result of tokenization is a list of individual strings that are
        words or their equivalent.

        Args:
            sentence (string):  The example sentence.
            sentiment (int):  The sentiment label.
        """

        # Get the relevant dicts for the sentiment
        s_word_counts = self.word_counts[sentiment]
        s_bigram_counts = self.bigram_counts[sentiment]
        s_bigram_denoms = self.bigram_denoms[sentiment]
        tokens = tokenize(sentence)
        for token in tokens:
            self.total_words[sentiment] += 1
            s_word_counts[token] = s_word_counts.get(token, 0) + 1
        my_bigrams = bigrams(tokens)
        for bigram in my_bigrams:
            s_bigram_counts[bigram] = s_bigram_counts.get(bigram, 0) + 1
            s_bigram_denoms[bigram[0]] = s_bigram_denoms.get(bigram[0], 0) + 1
            self.total_bigrams[sentiment] += 1
       
def get_models(df):
    """Returns a model_info object, consuming dataframe for examples."""
    last_fresh = 0
    info = ModelInfo()
    for _, line in df.iterrows():
        try:
            sentence_num = int(line['SentenceId'])
            if sentence_num <= last_fresh:
                continue
            last_fresh = sentence_num
            sentiment = int(line['Sentiment'])
            info.sentiment_counts[sentiment] += 1
            info.total_examples += 1
            info.update_word_counts(line['Phrase'], sentiment)
        except ValueError:
            # Some kind of bad input?  Unlikely with our provided data
            continue
    return info

In [7]:
info = get_models(df)
print(info)

words by sentiment: [20448, 42045, 29947, 45550, 24054] sentences: [1072, 2200, 1655, 2321, 1281]


In the following code, you just need to implement naive_bayes_classify() and markov_model_classify().  Notice the counts helpfully provided for you in the ModelInfo class; you should be able to calculate everything you need with those.

The "log probabilities" that these functions use and return aren't exactly true probabilities.  In a Naive Bayes classifier, we calculate numbers that are proportional to the true probabilities; the largest number corresponds to the most likely class.  So don't try to rescale these to sum to 1 across the sentiments; for larger texts to classify, this wouldn't work because of underflow.  And, speaking of underflow, please take the log of each term and add, rather than taking the log after multiplying everything out, since the whole point of taking logs is to avoid underflow, and waiting until after you've multiplied everything is too late.

The two key equations for the models are, for the Naive Bayes model,

$Pr(sentiment = c | words) \propto Pr(sentiment = c) \prod Pr(word_i |sentiment)$

and for the Markov model,

$Pr(sentiment = c | words) \propto Pr(sentiment = c) Pr(word_1 | sentiment = c) \prod Pr(word_i | word_{i-1}, sentiment = c)$

The "log probabilities" are the logs of the expressions on the right, although again, you should sum over logs instead of taking a log of the product.  All of these conditional probabilities can be calculated by counting.

In [11]:
""" Doing some Naive Bayes and Markov Models to do basic sentiment analysis.

Input from train.tsv.zip at
https:#www.kaggle.com/c/sentiment-analysis-on-movie-reviews

itself gathered from the Rotten Tomatoes movie review aggregation site.

Format is PhraseID[unused]   SentenceID  Sentence[tokenized] Sentiment

We'll only use the first line for each SentenceID, since the others are
micro-analyzed phrases that would just mess up our counts.

Sentiment is on a 5-point scale:
0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive

For each kind of model, we'll build one model per sentiment category.
Following Bayesian logic, base rates matter for each category; if critics
are often negative, that should be a good guess in the absence of other
information.

Training input is assumed to be in a file called "train.tsv"

Test sentences are received via stdin (thus either interactively or with input redirection).
Output for each line of input is the following:

Naive Bayes classification (0-4)
Naive Bayes most likely class's log probability (with default double digits/precision)
Markov Model classification (0-4)
Markov Model most likely class's log probability
"""

import sys
import math

CLASSES = 5
# Assume sentence numbering starts with this number in the file

# Probability of either a unigram or bigram that hasn't been seen -
# needs to be small enough that it's "practically a rounding error"
OUT_OF_VOCAB_PROB = 0.0000000001

def naive_bayes_classify(info, sentence):
    """ Use a Naive Bayes model to return sentence's most likely classification and the log prob.

    The log probability should be base e (natural log).

    Args:
        info (ModelInfo):  a ModelInfo containing the counts from the training data
        sentence (string):  the test sentence to classify

    Returns:
        int for the best sentiment
        float for the best log probability (unscaled, just ln(prior * product of cond. probs))
    """
    # Pr(sentiment=c|words)∝Pr(sentiment=c)∏Pr(wordi|sentiment)
    prob = []
    for x in info.sentiment_counts:
      prob.append(math.log(x / info.total_examples, math.e))

    words = tokenize(sentence)
    for i in words:
        for c in range(CLASSES):
            prob_i = info.word_counts[c].get(i, 0) / info.total_words[c]
            if (prob_i == 0):
                prob[c] += math.log(OUT_OF_VOCAB_PROB, math.e)
            else:
                prob[c] += math.log(prob_i, math.e)

    # Best sentiment, best log probability.
    return prob.index(max(prob)), max(prob)


def markov_model_classify(info, sentence):
    """ Like naive_bayes_classify, but use a bigram model for the evidence.

    The first word should still use a unigram probability to get started.
    Notice the existence of bigram_denoms, which has very slight differences from word_counts.
    Log probability is again base e.

    Args:
        info (ModelInfo):  a ModelInfo containing the counts from the training data
        sentence (string):  the test sentence to classify

    Returns:
        int for the best sentiment
        float for the best log probability (unscaled, just ln(prior * product of cond. probs))
    """
    # Pr(sentiment=c|words)∝Pr(sentiment=c)Pr(word1|sentiment=c)∏Pr(wordi|wordi−1,sentiment=c)
    prob = []
    for x in info.sentiment_counts:
      prob.append(math.log(x / info.total_examples, math.e))

    words = tokenize(sentence)
    my_bigrams = bigrams(words)
    i = words[0]
    for c in range(CLASSES):
        prob_i = info.word_counts[c].get(i, 0) / info.total_words[c]
        if prob_i == 0:
            prob[c] += math.log(OUT_OF_VOCAB_PROB, math.e)
        else:
            prob[c] += math.log(prob_i, math.e)

    # "Notic[ing] the existence of bigram_denoms."
    for bigram in my_bigrams:
        for c in range(CLASSES):
            counts = info.bigram_counts[c].get(bigram, 0)
            denoms = info.bigram_denoms[c].get(bigram[0], 0)
            if counts != 0:
                prob_bigrams = counts / denoms
            else:
                prob_bigrams = OUT_OF_VOCAB_PROB
            prob[c] += math.log(prob_bigrams, math.e)

    # Best sentiment, best log probability.
    return prob.index(max(prob)), max(prob)

Here are some sample sentences to test on, using the full dataset.  Note that it's possible to make minor mistakes, such as forgetting the prior, and still classify correctly, so just use this as a rough guide as to whether you're in the right ballpark.

In [13]:
test1 = "this is a terrible movie"  # expect 0, negative
test2 = "this is a delightful , joyous romp" # expect 4, very positive
test3 = "the dialogue stumbles occasionally , but usually delivers" # expect 2, neutral
# test4: expect 0 for basic nb, 2 for Markov (and true sentiment is more like a 4)
test4 = "I can't think of a single reason not to watch this movie"
print(naive_bayes_classify(info, test1))
print(naive_bayes_classify(info, test2))
print(naive_bayes_classify(info, test3))
print(naive_bayes_classify(info, test4))
print(markov_model_classify(info, test1))
print(markov_model_classify(info, test2))
print(markov_model_classify(info, test3))
print(markov_model_classify(info, test4))

(0, -27.002316729069577)
(4, -42.912159360571394)
(2, -56.43702265463601)
(0, -87.1044437764929)
(0, -17.643654407813663)
(4, -24.87612307805379)
(2, -107.25356901908081)
(2, -120.66613835938307)


If things are going wrong, it may be easier to use a small training file to test, where you can manually count and calculate probabilities.  The following example shows how to create a text test file in the notebook itself, converting it to a dataframe.  The expected sentiments aren't provided - the input is small enough that you could calculate the expected log probabilities with a calculator.  This input will also be used for grading.

In [14]:
from io import StringIO
testinput = """PhraseId\tSentenceId\tPhrase\tSentiment
1\t1\tamazing exciting movie hurrah\t4
2\t2\tjust amazing not sure how else to describe it\t4
3\t3\ta good time is guaranteed for everyone\t4
4\t4\tpretty good but not for everyone\t3
5\t5\tpretty clear this is not for everyone\t2
6\t6\tpretty thoroughly average movie\t2
7\t7\tnot sure why this movie was made\t1
8\t8\tthis movie should never have been made\t0
9\t9\tit is amazing how bad this movie is\t0"""

test_df = pd.read_table(StringIO(testinput))

In [15]:
test_df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,amazing exciting movie hurrah,4
1,2,2,just amazing not sure how else to describe it,4
2,3,3,a good time is guaranteed for everyone,4
3,4,4,pretty good but not for everyone,3
4,5,5,pretty clear this is not for everyone,2
5,6,6,pretty thoroughly average movie,2
6,7,7,not sure why this movie was made,1
7,8,8,this movie should never have been made,0
8,9,9,it is amazing how bad this movie is,0


In [16]:
test_info = get_models(test_df)
print(test_info)

words by sentiment: [15, 7, 11, 6, 20] sentences: [2, 1, 2, 1, 3]


In [17]:
test5 = "just amazing"
test6 = "not amazing"
test7 = "not for everyone"
test8 = "ugh"
print(naive_bayes_classify(test_info, test5))
print(naive_bayes_classify(test_info, test6))
print(naive_bayes_classify(test_info, test7))
print(naive_bayes_classify(test_info, test8))
print(markov_model_classify(test_info, test5))
print(markov_model_classify(test_info, test6))
print(markov_model_classify(test_info, test7))
print(markov_model_classify(test_info, test8))

(4, -6.396929655216146)
(4, -6.396929655216146)
(3, -7.5725029850203835)
(4, -24.124463218608568)
(4, -4.0943445622221)
(2, -26.9278235995151)
(2, -3.901972669574645)
(4, -24.124463218608568)


When you're confident your two methods are implemented correctly, **download the notebook and submit to Canvas**.

Sentiment analysis is a task that has many easy examples, but also hard examples that could stump even humans.  Naive Bayes is good for problems where you expect the relationship between the input and the classification to be relatively straightforward, or you're willing to tolerate incorrect classifications on some moderate to hard examples.  It's also very much on the easier side of ML to debug.